In [ ]:
import glob
import numpy as np
import matplotlib.pyplot as plt
import torch
import optuna
import os
import pytorch_lightning as pl
import yaml

from Data.Drosophilla.FlyDataMod import FlyDataModule
from IPython.core.debugger import set_trace
from Models import BiLSTM as bi
from torch import nn as nn
from Utils import callbacks as cb
from Utils import evaluations as ev
from Utils import HyperParams as hp
from Utils import loggers as lg

np.random.seed(0)

In [ ]:
def runBiLSTM_Exp(
    trial,
    label_type,
    label_val,
    lr,
    batch_size,
    num_layers,
    hidden_size,
    dropout,
    bias):
    logger = lg.DictLogger(trial.number,
                          root_dir)
    trainer = pl.Trainer(
        gpus=1,
        logger=logger,
        max_epochs=50,
        callbacks=[cb.getcb()],
        default_root_dir=root_dir)
    
    dm = FlyDataModule(cell_line="S2",
                     data_win_radius=5,
                     batch_size=batch_size,
                     label_type=label_type,
                     label_val=label_val)
    dm.setup()
    
    hparams = {'cell_line':"S2",
              "data_win_radius":5,
              "label_type":label_type,
              "batch_size":batch_size}
    
    model_bilstm = bi.BiLSTMModule(
                    input_size=29,
                    hidden_size=hidden_size,
                    num_layers=num_layers,
                    dropout=dropout,
                    bias=bias,
                    lr=lr,
                    hparams=hparams
                )
    model_bilstm.cuda()
    
    trainer.fit(model_bilstm, dm)
    score = logger.metrics[-1]['val weighted mse loss']
    if np.isnan(score):
        score = 99999999
    return score

def objective(trial):
    lr          = trial.suggest_categorical("lr", [1e-5, 1e-4, 1e-3, 1e-2, 1e-1])
    batch_size  = trial.suggest_categorical("batch_size", [1,4,16,64])
    num_layers  = trial.suggest_categorical("num_layers", [1,2,3,4,5,6])
    hidden_size = trial.suggest_categorical("hidden_size", [64])
    dropout     = trial.suggest_categorical("dropout", [0, 0.1, 0.2, 0.3])
    bias        = trial.suggest_categorical("bias", [True, False])
    print(lr, batch_size, num_layers, hidden_size, dropout, bias)
    return runBiLSTM_Exp(
        trial=trial,
        label_type=label_type,
        label_val=label_val,
        lr=lr,
        batch_size=batch_size,
        num_layers=num_layers,
        hidden_size=hidden_size,
        dropout=dropout,
        bias=bias)

In [ ]:
label_type = "insulation"
label_val  = 10
root_dir   = "Experiments/Table_8_LSTM_Tunning_Insulation"
if not os.path.isdir(root_dir):
    os.mkdir(root_dir)

In [ ]:
label_type = "insulation"
label_val  = 10
root_dir   = "Experiments/Table_8_LSTM_Tunning_Insulation"
if not os.path.isdir(root_dir):
    os.mkdir(root_dir)
study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=20)
hp.save_hyperparams(root_dir,
                   study.best_trial.number)

In [ ]:
#display table 8
label_type = "insulation"
label_val  = 10
dm         = FlyDataModule(cell_line="S2",
                          data_win_radius=5,
                          batch_size=1,
                          label_type=label_type,
                          label_val=label_val)
dm.setup()
vals = []

exps = sorted(glob.glob(root_dir+"/optuna/*"))
for e, exp in enumerate(exps):
    layer_weights = glob.glob(exp+"/checkpoints/*")[0]
    hparams       = yaml.full_load(open(glob.glob(exp+"/hparams.yaml")[0], 'r'))
    model         = bi.BiLSTMModule.load_from_checkpoint(layer_weights).to("cuda:0")
    row           = []
    row.append(hparams['lr'])
    row.append(hparams['hparams']['batch_size'])
    row.append(hparams['num_layers'])
    row.append(hparams['dropout'])
    row.append(hparams['bias'])
    mm            = ev.getModelMetrics(model, dm, 'val').values()
    fmm           = list(map(lambda x: "{:.2f}".format(x), mm ))
    row.extend(fmm)
    vals.append(row)
cols = ['lr','b_sz', '# lyrs', 'dropout','bias','val_mse','val_mae','val_r2','val_pcc','val_spc']
fig, ax = plt.subplots(1, figsize=(20,20))
table   = ax.table(vals,
                  cellLoc="center",
                  colLabels=cols,
                  rowLabels=list(range(0, len(exps))))
table.set_fontsize(14)
ax.axis('off')
plt.show()

In [ ]:
dm = FlyDataModule(cell_line="S2",
                  data_win_radius=5,
                  batch_size=1,
                  label_type="insulation",
                  label_val=10)
dm.setup()
layer_weights = glob.glob("Experiments/Table_8_LSTM_Tunning_Insulation/optuna/version_6/checkpoints/*")[0]
print(layer_weights)
model         = bi.BiLSTMModule.load_from_checkpoint(layer_weights).to("cuda:0")
ev.createPlot(model, dm, "train", "idk")

In [ ]:
#to csv
import pandas as pd
ar_val   = np.array(vals)
ar_col   = np.expand_dims(np.array(cols),0)
csv_data = np.vstack((ar_col, ar_val))
pd.DataFrame(csv_data).to_csv("Experiments/Table_8_LSTM_Tunning_Insulation/Sup_8_Results.csv")